In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

/Users/unger/dev/src/mountaineersdata/env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
class FileCols(object):
	# Class holds information about a file.
	def __init__(self, allCols, dateCols, intCols, dropCols):
		self.allCols = allCols
		self.dateCols = dateCols
		self.intCols = intCols
		self.dropCols = dropCols

	@property 
	def colTypeMap(self):
		tm = {}
		for n in self.allCols:
			if not n in self.dateCols:
				if n in self.intCols:
					tm[n] = int
				else:
					tm[n] = str
		return tm

	@property
	def fixnaCols(self):
		# A list of columns where empty values should be changed to empty string
		return [ n for n in self.allCols if n not in self.dropCols and n not in self.dateCols ]

DataDirectory = Path(Path.home(), "Documents/mountaineers/kayak-seattle/data/paddleractivity")

# Define contents of our two data files
# Roster contains a roster of graduated students.
RosterFile = FileCols([ "First Name", "Last Name", "Email", "Registration Status", "Course Position", "Graduation Date" ],
								[ "Graduation Date"] , [], ["Graduation Date"])
# Activity contains a list of graduate's subsequent activities
ActivityFile = FileCols([ "First Name", "Last Name", "Email", "Course Title", "Activity Title", "Activity Category", "Activity Type", "Activity Start Date", "Activity End Date", "Activity Result", "Activity Participant Registration Status", "Activity Participant Result" ],
								["Activity Start Date", "Activity End Date"], [], [ "Activity Result", "Activity Participant Result"])

EquivRosterFileBase = "Seattle Equivalency"
EquivActivityFileBase = "Seattle Equivalency Activity"
EquivDates = [ 2015, 2016, 2021 ]

BasicRosterFileBase = "Seattle Basic"
BasicActivityFileBase = "Seattle Basic Activity"
BasicDates = [ 2015, 2016, 2017, 2018, 2019, 2021 ]

# Types of activities we count.  Exclude field trips, lectures, and seminars. 
ActivityTypeList = [ "Trip", "Clinic" ]



In [7]:
def LoadData (dataDirectory, rosterFileBase, rosterCols: FileCols, activityFileBase, activityCols: FileCols, paddlerType, dateList, activityTypeList):
	roster = None
	activity = None
	for date in dateList:
		#
		# Load the roster
		#
		fileName = f"{rosterFileBase} {date}.csv"
		print (f"++++ {fileName} ++++")
		filePath = Path(dataDirectory, fileName)
		ctm = rosterCols.colTypeMap
		r = pd.read_csv(filePath, usecols=rosterCols.allCols, parse_dates=rosterCols.dateCols, dtype=ctm)
		r.info()
		# Drop people who did not graduate
		r = r[r["Course Position"].isin(["Graduate", "Equivalent"])]

		# Fix columns that should contain empty string, not NaN and drop rows that still have NaN
		for n in rosterCols.fixnaCols:
			r[n].fillna('')
		r.dropna(inplace=True)


		if roster is None:
			roster = r
		else:
			roster = roster.append(r)


		#
		# Load the activity report
		#
		fileName = f"{activityFileBase} {date}.csv"
		filePath = Path(dataDirectory, fileName)
		a = pd.read_csv(filePath, usecols=activityCols.allCols, parse_dates=activityCols.dateCols, dtype=activityCols.colTypeMap)
		a.info()

		# Fix columns that should contain empty string, not NaN and drop rows that still have NaN
		for n in activityCols.fixnaCols:
			a[n].fillna('')
		a.dropna(inplace=True)

		# Limit to Sea Kayaking and specific activity category
		a = a[ (a["Activity Type"] == "Sea Kayaking") ]
		a = a[ a["Activity Category"].isin(activityTypeList) ]
		if activity is None:
			activity = a
		else:
			activity = activity.append(a)

		

	# Set paddler type and full name field for easier cross reference.
	roster["Paddler Type"] = paddlerType
	roster["Full Name"] = roster["First Name"] + " " + roster["Last Name"]
	activity["Full Name"] = activity["First Name"] + " " + activity["Last Name"]

	print ("Roster before return")
	roster.info()

	return (roster, activity)

(basicRoster, basicActivity) = LoadData(DataDirectory, BasicRosterFileBase, RosterFile, BasicActivityFileBase, ActivityFile, "Basic", BasicDates, ActivityTypeList)
#basicRoster.to_csv("BasicRoster.csv")
#basicActivity.to_csv("BasicActivity.csv")
(equivRoster, equivActivity) = LoadData(DataDirectory, EquivRosterFileBase, RosterFile, EquivActivityFileBase, ActivityFile, "Equivlency", EquivDates, ActivityTypeList)
#equivRoster.to_csv("EquivRoster.csv")
#equivActivity.to_csv("EquivActivity.csv")



++++ Seattle Basic 2015.csv ++++
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   First Name           44 non-null     object        
 1   Last Name            44 non-null     object        
 2   Email                44 non-null     object        
 3   Registration Status  44 non-null     object        
 4   Course Position      44 non-null     object        
 5   Graduation Date      32 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 2.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   First Name                                916 non-null    object        
 1   Last N

In [ ]:
# Calculate trip counts for entries on roster
equivRoster["Trip Count"] = equivRoster.apply(lambda row : len(equivActivity.loc[equivActivity['Full Name'] == row['Full Name']]), axis=1)
basicRoster["Trip Count"] = basicRoster.apply(lambda row : len(basicActivity.loc[basicActivity['Full Name'] == row['Full Name']]), axis=1)

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True, figsize=(30,12))
b = max(equivRoster["Trip Count"].max(), basicRoster["Trip Count"].max())
ax[0].hist(equivRoster["Trip Count"], bins=b)
ax[0].set_title("Equivlency Paddlers")
ax[1].hist(basicRoster["Trip Count"], bins=b)
ax[1].set_title("Basic Class Paddlers")


In [ ]:
plt.figure(figsize=(25,10))
b = max(equivRoster["Trip Count"].max(), basicRoster["Trip Count"].max())
plt.hist(basicRoster["Trip Count"], bins=b, alpha=1.0, label="Basic")
plt.hist(equivRoster["Trip Count"], bins=b, alpha=0.7, label="Equivlency")
plt.xlabel("Number of Trips")
plt.xticks(range(0,b,5))
plt.ylabel("Number of Paddlers")
plt.legend()


In [ ]:
equivRoster.info()

In [23]:
# def groupYear(x: pd.Timestamp):
# 	if isinstance (x, pd.Timestamp):
# 		print (x.year)
# 		return x.year
# 	return None
#by = bs.groupby(groupYear).sum()

bs = pd.Series(1,index=basicRoster['Graduation Date'])
by = bs.groupby(pd.Grouper(freq='YS')).sum()

ey = pd.Series(1,index=equivRoster['Graduation Date']).groupby(pd.Grouper(freq='YS')).sum()
ey.to_csv("EquivByYear.csv")
plt.bar(ey["Graduation Date"], ey[1])


# from https://www.tutorialspoint.com/matplotlib/matplotlib_bar_plot.htm
data = [[30, 25, 50, 20],
[40, 23, 51, 17],
[35, 22, 45, 19]]
X = np.arange(4)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, data[1], color = 'g', width = 0.25)
ax.bar(X + 0.50, data[2], color = 'r', width = 0.25)